In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import zipfile
from skimage import color
from matplotlib import pyplot
import keras
from keras.layers import  Conv2D, MaxPooling2D, BatchNormalization, \
    GlobalAveragePooling2D, Dense, Input
from keras.layers import Flatten,Dropout
from keras.models import Sequential
from keras.optimizers import SGD,Adam
from keras.initializers import glorot_uniform
import time

%matplotlib inline

In [0]:
# Import data and label files

# Train labels
training_labels_downloaded = drive.CreateFile({'id': '1rJCglyD-6uTow7RFsj5K7WxSl1YFyg8M'})
training_labels_downloaded.GetContentFile('train.txt')
train_labels = pd.read_csv('train.txt', sep=" ", header=None)

# Test labels
test_labels_downloaded = drive.CreateFile({'id': '15a5U1SsguFSdQim7ifoA9vpuhyL5AgTZ'})
test_labels_downloaded.GetContentFile('vali.txt')
test_labels = pd.read_csv('vali.txt', sep=" ", header=None)

# Train data
train_data_downloaded = drive.CreateFile({'id': '1-X94bMCmjMShVVN5VJXPZ936Zly0BOP_'})
train_data_downloaded.GetContentFile('train-set.zip')
train_data = zipfile.ZipFile('train-set.zip', 'r')
train_data.extractall()

# Test data
test_data_downloaded = drive.CreateFile({'id': '1lSMxDwZFD40ceFAlWCoqT1TlJ9gqgqOx'})
test_data_downloaded.GetContentFile('vali-set.zip')
test_data = zipfile.ZipFile('vali-set.zip', 'r')
test_data.extractall()

In [0]:
%%time
### Import and convert data images into RGB arrays ###
import scipy.misc

# Train data
train_images = []
for i in range (0, len(train_labels)):
  im = scipy.misc.imread('train-set/train-set/'+train_labels[0][i], flatten=False, mode='L')
  train_images.append(im.astype('uint8'))
   #train_images.append(cv2.imread('train/train-set/'+train_labels[0][i]).astype('uint8'))

# Test data
test_images = []
for i in range (0, len(test_labels)):
  im = scipy.misc.imread('vali-set/vali-set/'+test_labels[0][i], flatten=False, mode='L')
  test_images.append(im.astype('uint8'))

# Define train and test sets
y_train = train_labels[1][:]
y_test = test_labels[1][:]
x_train = (np.asarray(train_images)).reshape(len(train_images), len(train_images[0]), len(train_images[0]), 1)
x_test = (np.asarray(test_images)).reshape(len(test_images), len(test_images[0]), len(test_images[0]), 1)


class_num = len(np.unique(y_train))


# One-hot code the labels
y_train = keras.utils.to_categorical(y_train,class_num)
y_test = keras.utils.to_categorical(y_test,class_num)

# Print shapes of data and labels
print('Train data: ',x_train.shape)
print('Train labels: ',y_train.shape)
print('Test data: ',x_test.shape)
print('Test labels: ',y_test.shape)

In [0]:

##start a sequential   
zfmodel = Sequential()  
#the first convoultion layer
zfmodel.add(Conv2D(96,(7,7),strides=(2,2),input_shape=(128,128,1),padding='valid',activation='relu',kernel_initializer= glorot_uniform(seed=8)))  
zfmodel.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
#The second convoultion layer
zfmodel.add(Conv2D(256,(5,5),strides=(2,2),padding='same',activation='relu',kernel_initializer= glorot_uniform(seed=6)) ) 
zfmodel.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
#the third convoultion layer
zfmodel.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer= glorot_uniform(seed=1)))
#the fouth convoultion layer
zfmodel.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer= glorot_uniform(seed=2)))
#the fifth convoultion layer
zfmodel.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer= glorot_uniform(seed=3)))  
zfmodel.add(MaxPooling2D(pool_size=(3,3),strides=(2,2))) 
#the faltten 
zfmodel.add(Flatten())  
#three fully connected layers with dropout that avoiding overfitting 
zfmodel.add(Dense(4096,activation='relu'))  
zfmodel.add(Dropout(0.5))  
zfmodel.add(Dense(4096,activation='relu'))  
zfmodel.add(Dropout(0.5))  
zfmodel.add(Dense(62,activation='softmax'))  
##Keep a history track 
history = zfmodel.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.001,momentum=0.9, decay = 1e-6),
              metrics=['accuracy'])
zfmodel.summary()  


In [0]:
# Check runtime
start = time.time()

# Train model

zfmodel.fit(x_train,y_train,epochs=20,batch_size = 128,
         validation_data=(x_test, y_test),
         verbose=2)
score = zfmodel.evaluate(x_test, y_test, verbose=0)
print('Test loss: {:0.2f}%'.format(score[0]*100))
print('Test accuracy: {:0.2f}%'.format(score[1]*100))

# Print runtime
end = time.time()
print('Total runtime is {:0.2f} minutes'.format(int(end-start)/60))